In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
import selenium
from selenium import webdriver
import time
import io
import requests
import re
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException

In [2]:
#Specify Search URL 

city = 'milano'
search_url = "https://www.immobiliare.it/vendita-case/{}/?criterio=rilevanza".format(city)

In [3]:
def connect(web_addr, page_number = None):
    if page_number == 1:
        resp = requests.get(web_addr)
    else:
        resp = requests.get(web_addr+'&pag={}'.format(page_number))
    return BeautifulSoup(resp.content, "html.parser")

#data = connect(search_url, 2)

In [4]:
df = pd.DataFrame(columns = ['Address', 'Link'])

for page in range(1,20): #scrape over the first 20 pages
    addresses = []
    links = []
    data = connect(search_url, page)
    for ann in data.find_all('a', {'class': 'in-card__title'}):
        addresses.append(ann['title'])
        links.append(ann['href'])
        df = pd.concat([df,pd.DataFrame({'Address':addresses, 'Link': links})], axis = 0)

df = df.drop_duplicates()

6175


,Address,Link
0,Appartamenti di nuova costruzione in vendita a...,https://www.immobiliare.it/annunci/p-157721/
0,Appartamenti di nuova costruzione in vendita a...,https://www.immobiliare.it/annunci/p-157721/
1,Appartamenti di nuova costruzione in vendita a...,https://www.immobiliare.it/annunci/p-157413/
0,Appartamenti di nuova costruzione in vendita a...,https://www.immobiliare.it/annunci/p-157721/
1,Appartamenti di nuova costruzione in vendita a...,https://www.immobiliare.it/annunci/p-157413/


In [6]:
def scrape_house_details(url):
    data = connect(url)
    info = data.find_all('dl', {'class': 'im-features__list'}) 
    info_address = data.find_all('div', {'class': 'im-map__description'})
    addr = re.sub('\n', ',', info_address[0].get_text().strip())
    
    db = pd.DataFrame()
    
    address = []
    attributes = []
    for n in range(len(info)):
        for i in info[n].find_all('dt'):
            address.append(i.text)
        for i in info[n].find_all('dd'):
            attributes.append(i.text)
    
    db['Id'] = address
    db['Attribute'] = attributes
    
    feature = []
    for item in db['Attribute']:
        try:
            feature.append(fix_strings(item))
        except:
            feature.append(fix_strings2(item))

            db['Attribute'] = feature
    return np.append(np.array('indirizzo'),db['Id'].values), np.append(np.array(addr),db['Attribute'].values)
    

def remove_duplicates(x):
    return list(dict.fromkeys(x))

def fix_strings(l):
    l = (l.split('\n')[1].split('  '))
    l = [i for i in l if i != ''][0]
    return the_list

def fix_strings2(l):
    l = (l.split('\n\n')[1].split('  '))
    l = [i for i in l if i != ''][0]
    l = (l.split('\n')[0])
    return the_list

In [7]:
#scrape_house_details('https://www.immobiliare.it/annunci/p-157721/')

In [8]:
columns=['indirizzo','riferimento e Data annuncio', 'contratto', 'tipologia',
        'superficie', 'locali', 'piano', 'totale piani edificio',
        'disponibilità', 'Tipo proprietà', 'altre caratteristiche',
        'prezzo', 'spese condominio', 'anno di costruzione', 'stato',
        'riscaldamento', 'Efficienza energetica', 'indirizzo']
for col in columns:
    df[col] = ''
    
df.set_index('Link', inplace = True)

In [9]:
import re
for link in df.index:
    titles, attributes = scrape_house_details(link)
    for i in range(len(titles)):
        df.loc[link, titles[i]] = attributes[i]
df

,Address,indirizzo,riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,...,riscaldamento,Efficienza energetica,unità,Data di inizio lavori e di consegna prevista,Climatizzatore,immobile garantito,Posti Auto,Prestazione energetica del fabbricato,Indice prest. energetica rinnovabile,certificazione energetica
Link,,,,,,,,,,,,,,,,,,,,,
https://www.immobiliare.it/annunci/p-157721/,Appartamenti di nuova costruzione in vendita a...,"Milano,Carrobbio,Via Borromei, 9",EK-157721 - 25/10/2021,Vendita,,,,,,,...,,F ≥ 175 kWh/m² anno,12 abitative,01/07/2020 - 31/12/2021,NaN,NaN,NaN,NaN,NaN,NaN
https://www.immobiliare.it/annunci/p-157413/,Appartamenti di nuova costruzione in vendita a...,"Milano,Via Garofalo, 10",EK-157413 - 30/11/2021,Vendita,,,,,,,...,,"A2 ≥ 3,51 kWh/m² anno",97 abitative,01/07/2021 - 01/09/2023,NaN,NaN,NaN,NaN,NaN,NaN
https://www.immobiliare.it/annunci/91836408/,"Quadrilocale via Pietro Panzeri 15, Ticinese, ...","Milano,Ticinese,Via Pietro Panzeri, 15",EK-91836408 - 25/11/2021,Vendita,Appartamento,167 m²,"4 (3 camere da letto, 1 altro), 2 bagni, cucin...","Piano terra, con ascensore",6 piani,Libero,...,"Autonomo, ad aria, alimentazione elettrica",G ≥ 175 kWh/m² anno,NaN,NaN,"Autonomo, freddo/caldo",NaN,NaN,NaN,NaN,NaN
https://www.immobiliare.it/annunci/p-162426/,Appartamenti di nuova costruzione in vendita a...,"Milano,Ripamonti,Via Adamello, 10",EK-162426 - 22/11/2021,Vendita,,,,,,,...,,A ≥ 175 kWh/m² anno,83 abitative,02/11/2021 - 31/03/2024,NaN,NaN,NaN,NaN,NaN,NaN
https://www.immobiliare.it/annunci/p-161603/,Appartamenti di nuova costruzione in vendita a...,"Milano,Via Alessandro Manzoni, 40",EK-161603 - 24/11/2021,Vendita,,,,,,,...,,F ≥ 175 kWh/m² anno,7 abitative,01/06/2021 - 01/12/2021,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.immobiliare.it/annunci/91198118/,"Trilocale via Val Gardena 25, Precotto, Milano","Milano,Precotto,Via Val Gardena, 25",Val Gardena - 15/11/2021,Vendita,Appartamento,80 m²,"3 (2 camere da letto, 1 altro), 1 bagno, cucin...",3°,3 piani,Libero,...,"Autonomo, a radiatori, alimentato a gas","G 241,42 kWh/m² anno",NaN,NaN,"Autonomo, caldo","Dati certificabili, documentazione completa",NaN,NaN,NaN,NaN
https://www.immobiliare.it/annunci/89655255/,"Bilocale via Marghera, Vercelli - Wagner, Milano","Milano,Vercelli - Wagner,Via Marghera",W-02MG19 - 14/09/2021,Vendita,Appartamento,52 m²,"2 (1 camera da letto, 1 altro), 1 bagno, cucin...",3°,3 piani,Libero,...,"Autonomo, a radiatori, alimentato a gas","D 105,82 kWh/m² anno",NaN,NaN,"Autonomo, freddo/caldo","Dati certificabili, documentazione completa",NaN,NaN,NaN,NaN
https://www.immobiliare.it/annunci/88606147/,"Bilocale via Palanzone 18, Niguarda, Milano","Milano,Niguarda,Via Palanzone 18",PAL18,Vendita,Appartamento,53 m²,"2 (1 camera da letto, 1 altro), 2 bagni, cucin...","4° piano, con ascensore, con accesso disabili",4 piani,Libero,...,"Autonomo, a radiatori, alimentato a metano","C 134,00 kWh/m² anno",NaN,NaN,NaN,"Dati certificabili, documentazione completa",NaN,NaN,NaN,NaN


In [10]:
df.rename({'Address': 'Title'}, axis = 1, inplace = True)

In [11]:
#Not very useful cols, many nulls
df.drop(['certificazione energetica', 'Indice prest. energetica rinnovabile',
        'Prestazione energetica del fabbricato','unità',
        'Data di inizio lavori e di consegna prevista'], axis = 1, inplace = True)

In [12]:
#Drop also announcements with no details about surface
df.drop(df[df.superficie == ''].index, axis = 0, inplace = True)
df.head()

,Title,indirizzo,riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,...,altre caratteristiche,prezzo,spese condominio,anno di costruzione,stato,riscaldamento,Efficienza energetica,Climatizzatore,immobile garantito,Posti Auto
Link,,,,,,,,,,,,,,,,,,,,,
https://www.immobiliare.it/annunci/91836408/,"Quadrilocale via Pietro Panzeri 15, Ticinese, ...","Milano,Ticinese,Via Pietro Panzeri, 15",EK-91836408 - 25/11/2021,Vendita,Appartamento,167 m²,"4 (3 camere da letto, 1 altro), 2 bagni, cucin...","Piano terra, con ascensore",6 piani,Libero,...,Cancello elettrico,€ 560.000,€ 250/mese,1960,Ottimo / Ristrutturato,"Autonomo, ad aria, alimentazione elettrica",G ≥ 175 kWh/m² anno,"Autonomo, freddo/caldo",NaN,NaN
https://www.immobiliare.it/annunci/92317086/,"Quadrilocale via Francesco Primaticcio 140, Ba...","Milano,Bande Nere,Via Francesco Primaticcio 140",00324 - 29/11/2021,Vendita,Appartamento,130 m²,"4 (3 camere da letto, 1 altro), 2 bagni, cucin...","7°, con ascensore, con accesso disabili",8 piani,,...,Cancello elettrico,€ 564.000,€ 333/mese,1980,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano","G 175,00 kWh/m² anno","Autonomo, freddo","Dati certificabili, documentazione completa",1 in garage/box
https://www.immobiliare.it/annunci/86462824/,"Trilocale via Ruggiero Settimo, Washington, Mi...","Milano,Washington,Via Ruggiero Settimo",T3212I - 11/11/2021,Vendita,Appartamento,60 m²,"3 (2 camere da letto, 1 altro), 1 bagno, cucin...","1°, con ascensore",8 piani,,...,Porta blindata,€ 420.000,€ 120/mese,1970,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a metano","G 224,41 kWh/m² anno","Autonomo, freddo/caldo","Dati certificabili, documentazione completa",NaN
https://www.immobiliare.it/annunci/92254762/,"Bilocale via Paolo Lomazzo 13, Paolo Sarpi, Mi...","Milano,Paolo Sarpi,Via Paolo Lomazzo, 13",LOM13 - 29/11/2021,Vendita,Appartamento,50 m²,"2 (1 camera da letto, 1 altro), 1 bagno, cucin...",Piano terra,4 piani,Libero,...,Fibra ottica,€ 250.000,€ 90/mese,1920,Buono / Abitabile,"Autonomo, a radiatori, alimentato a metano",F ≥ 175 kWh/m² anno,"Autonomo, freddo/caldo","Dati certificabili, documentazione completa",NaN
https://www.immobiliare.it/annunci/90552533/,"Bilocale piazza Santa Maria Beltrade 1, Carrob...","Milano,Carrobbio,Piazza Santa Maria Beltrade, 1",T21047 - 06/10/2021,Vendita,Appartamento,53 m²,"2 (1 camera da letto, 1 altro), 1 bagno, cucin...","2°, con ascensore, con accesso disabili",7 piani,Libero,...,Fibra ottica,€ 750.000,€ 250/mese,2016,Ottimo / Ristrutturato,"Centralizzato, a pavimento, alimentato a metano","B 37,48 kWh/m² anno","Autonomo, freddo","Dati certificabili, documentazione completa",NaN


In [17]:
import names
names.get_full_name()

'Antonia Chobot'

In [21]:
df['Owner'] = ''*len(df)
for i in df.index:
    df.loc[i,'Owner'] = names.get_full_name()

In [13]:
df.to_csv('LandRegistry.csv')